In [10]:
import ast
import numpy as np
import time
import sys
import pandas as pd
import subprocess
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
%pdb

Automatic pdb calling has been turned OFF


In [2]:
# Loading kmer table
with open('data/km_table_0.py', 'rb') as f:
    kmer_table = pickle.load(f)

In [4]:
# Loading patient table
patient_table = pd.read_csv('data/API_getTable_715c0c.csv')
patient_table.columns = ["id", "fab", "prog", "sex", "who"]

In [5]:
# Extracting inv(16) label
inv_16 = [("inv(16)" in text) for text in patient_table.who[:-1]]
inv_16 = [(1 if i else 0) for i in inv_16]

In [6]:
# Splitting data
x_train, x_test, y_train, y_test = train_test_split(kmer_table, inv_16, random_state=4)

In [7]:
# Data pre-processing/transformations
sc = StandardScaler()
sc.fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)

In [8]:
# Define classifier
classifier = MLPClassifier(hidden_layer_sizes=(1000,500,250,125,50), max_iter=300, activation = 'relu', solver='adam', random_state=1)

In [13]:
# Train model
classifier.fit(x_train, y_train)

MLPClassifier(hidden_layer_sizes=(1000, 500, 250, 125, 50), max_iter=300,
              random_state=1)

In [14]:
# Predict & accuracy
y_pred = classifier.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9248554913294798


In [25]:
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [24]:
print(np.array(y_test))

[0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
